---

# DS400: Assignment V: GitHub and the ticketmaster.com API (Python)

##### Max Mohr (Student ID: 6304784)
##### January 31, 2023

*I hereby acknowledge that the submitted assignment is my own work. During the preparation of this assignment I have worked together with Max Mohr and Felix Koehn.*

GitHub Repository: https://github.com/maxjmohr/MSc_01_DSPM_Assignment05

In [27]:
# set working directory
import os
os.chdir("/Users/mAx/Documents/Master/Assignment_05/")

In [51]:
# load packages
import geopandas as gpd
import requests
import pandas as pd
import io

---

### Setting up a new GitHub repository

<div class="alert alert-block alert-info">
    
1. Register on github.com in case you have not done this already.
</div>

<div class="alert alert-block alert-info">
    
2. Initialize a new public repository for this assignment on GitHub.
</div>

<div class="alert alert-block alert-info">
    
3. For the following exercises of this assignment, follow the standard Git workflow (i.e., pull the latest
version of the project to your local computer, then stage, commit, and push all the modifications that you make throughout the project). Every logical programming step should be well documented on GitHub with a meaningful commit message, so that other people (e.g., your course instructor) can follow and understand the development history. You can do this either using Shell commands or a Git GUI of your choice.
</div>

<div class="alert alert-block alert-info">

4. In the HTML file that you submit, include the hyperlink to the project repository (e.g., https://github. com/yourUserName/yourProjectName)
</div>

---

### Getting to know the API

<div class="alert alert-block alert-info">
    
5. Visit the documentation website for the API provided by ticketmaster.com (see here). Familiarize your- self with the features and functionalities of the Ticketmaster Discovery API. Have a particular look at rate limits.
</div>

<div class="alert alert-block alert-info">

6. Whithin the scope of this assignment, you do not have to request your own API key. Instead retrieve a valid key from the API Explorer. This API key enables you to perform the GET requests needed throughout this assignment. Even though this API key is not secret per se (it is publicly visible on the API Explorer website), please comply to the common secrecy practices discussed in the lecture and the tutorial: Treat the API key as a secret token. Your API key should neither appear in the code that you are submitting nor in your public GitHub repository.
</div>

In [15]:
# access the key from txt file
with open('api_keys/ticketmaster_api_key.txt') as script:
    ticketmaster_api_key = script.read()

---

### Interacting with the API - the basics

<div class="alert alert-block alert-info">
    
7. Perform a first GET request, that searches for event venues in Germany (countryCode = "DE"). Extract the content from the response object and inspect the resulting list. Describe what you can see.
</div>

In [26]:
# perform get request
api_content = requests.get("https://app.ticketmaster.com/discovery/v2/venues?",
                            params = {"apikey": ticketmaster_api_key,
                                      "countryCode": "DE"
                                      }).content


api_content

b'{"_embedded":{"venues":[{"name":"Gruenspan","type":"venue","id":"KovZpZAneakA","test":false,"url":"http://www.ticketmaster.de/venue/287155","locale":"en-de","images":[{"ratio":"16_9","url":"https://s1.ticketm.net/dbimages/2057v.","width":205,"height":115,"fallback":false}],"postalCode":"22767","timezone":"Europe/Berlin","city":{"name":"Hamburg"},"country":{"name":"Germany","countryCode":"DE"},"address":{"line1":"Grosse Freiheit 58"},"location":{"longitude":"9.958075","latitude":"53.551885"},"markets":[{"name":"Germany","id":"210"}],"dmas":[{"id":610}],"boxOfficeInfo":{"phoneNumberDetail":"Gruenspan Gro\xc3\x9fe Freiheit 58 22767 Hamburg Tel: 040-313616 mail: info@gruenspan.de web: www.gruenspan.de"},"upcomingEvents":{"_total":2,"mfx-de":2,"_filtered":0},"ada":{"adaPhones":"+49.(0)1805 - 969 0000 (14 Ct./Min.)","adaCustomCopy":"Sollten Sie besondere Sitzplatzw\xc3\xbcnsche haben, pr\xc3\xbcfen Sie bitte die Informationsseite des Veranstaltungsortes. F\xc3\xbcr weitergehende Informatio

By briefly looking at the content of the get request, we are able to see some of the venues in Germany. At least regarding the few examples we are able to see, the type is indeed a "venue". Furthermore, te countryCode is "DE".

However, not all venues have the exact same information displayed in each column. For example, one venue named "Virtual event" doesn't have any data for the column "address". Another example is the venue named "Huxleys Neue Welt" has an image while others don't.

We in total have 4.745 elements that are German venues, displayed over 238 pages. One page displays 20 results.

<div class="alert alert-block alert-info">

8. Extract the name, the city, the postalCode and address, as well as the url and the longitude and latitude of the venues to a data frame. This data frame should have the following structure (see pdf).
</div>

In [215]:
venues = requests.get("https://app.ticketmaster.com/discovery/v2/venues?",
                        params = {"apikey": ticketmaster_api_key,
                                  "countryCode": "DE"
                                  }).json()

# The data we are looking for lies in _embedded/venues
venues = venues['_embedded']['venues']

# Select the columns ["name","city","postalCode","address","url","location"]
# If a venue doesn't have a certain column, the column will be filled with a None value
venues_name       = [v['name'] if ('name' in v.keys()) else None for v in venues]
venues_city       = [v['city']['name'] if ('city' in v.keys()) else None for v in venues]
venues_postalCode = [v['postalCode'] if ('postalCode' in v.keys()) else None for v in venues]
venues_address    = [v['address']['line1'] if ('address' in v.keys()) else None for v in venues]
venues_url        = [v['url'] if ('url' in v.keys()) else None for v in venues]
venues_longitude  = [v['location']['longitude'] if ('location' in v.keys()) else None for v in venues]
venues_latitude   = [v['location']['latitude'] if ('location' in v.keys()) else None for v in venues]

# Create dataframe
venues_df = pd.DataFrame({'name': venues_name,
                          'city': venues_city,
                          'postalCode': venues_postalCode,
                          'address': venues_address,
                          'url': venues_url,
                          'longitude': venues_longitude,
                          'latitude': venues_latitude
                          })

# Change the formats of some columns
venues_df = venues_df.astype({'longitude':float})
venues_df = venues_df.astype({'latitude':float})

venues_df
# venues_df.dtypes  # to check the data types of columns

,name,city,postalCode,address,url,longitude,latitude
0,Gruenspan,Hamburg,22767,Grosse Freiheit 58,http://www.ticketmaster.de/venue/287155,9.958075,53.551885
1,Huxleys Neue Welt,Berlin,10967,Hasenheide 107 – 113,http://www.ticketmaster.de/venue/286842,13.421380,52.486391
2,Virtual Event,Worldwide,None,None,https://www.ticketmaster.de/venue/virtuelles-e...,10.000000,50.000000
3,Ev. St. Jacobi Kirche,Sangerhausen,06526,Marktplatz,http://www.ticketmaster.de/venue/290061,NaN,NaN
4,Evangelische Kirche,Senden,48308,Steverstrasse 5,http://www.ticketmaster.de/venue/290066,NaN,NaN
5,HDI Arena,Hannover,None,Robert-Enke-Straße 1,http://www.ticketmaster.de/venue/461692,9.733710,52.361993
6,Arsenal,Berlin,10785,Potsdamer Strasse 2,http://www.ticketmaster.de/venue/290646,NaN,NaN
7,Freilichtbühne Heppenheim,Heppenheim,64646,Oberhalb der Stadt,http://www.ticketmaster.de/venue/290639,NaN,NaN
8,Schlosswallhalle,Osnabrück,49074,Schlosswall 10,http://www.ticketmaster.de/venue/290630,NaN,NaN
9,Metropol Theater,Vechta,49377,Kolpingstrasse 27,http://www.ticketmaster.de/venue/290631,NaN,NaN


---

### Interacting with the API - advanced

<div class="alert alert-block alert-info">
    
9. Have a closer look at the list element named page. Did your GET request from exercise (7) return all event locations in Germany? Obviously not - there are of course much more venues in Germany than those contained in this list. Your GET request only yielded the first results page containing the first 20 out of several thousands of venues. Check the API documentation under the section Venue Search. How can you request the venues from the remaining results pages? Iterate over the results pages and perform GET requests for all venues in Germany. After each iteration, extract the seven variables name, city, postalCode, address, url, longitude, and latitude. Join the information in one large data frame. Print the first 10 rows and the shape of the resulting data frame. The resulting data frame should look something like this (note that the exact number of search results may have changed since this document has been last modified) (see pdf).
</div>

---

### Visualizing the extracted data

<div class="alert alert-block alert-info">
    
10. Below, you can find code that produces a map of Germany. Add points to the map indicating the locations of the event venues across Germany.
</div>

In [ ]:
# get a base map of Germany
map = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
map = map[map.name == "Germany"]
# plot the map
map.plot(ax = ax)

<div class="alert alert-block alert-info">

11. You will find that some coordinates lie way beyond the German borders and can be assumed to be faulty. Set coordinate values to NA where the value of longitude is outside the range (5.866, 15.042) or where the value of latitude is outside the range (47.270, 55.059) (these coordinate ranges have been derived from the extreme points of Germany as listed on Wikipedia (see here). For extreme points of other countries, see here).
</div>

---

### Event locations in other countries

<div class="alert alert-block alert-info">
    
12. Repeat exercises (9)–(11) for another European country of your choice. (Hint: Clean code pays off! If you have coded the exercises efficiently, only very few adaptions need to be made.)
</div>